<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Изучение данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#NB-SVM" data-toc-modified-id="NB-SVM-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>NB-SVM</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LinearSVC</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Машинное обучение для текстов
# Проект "Поиск токсичных комментариев"
# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

**Цели проекта**: 
 - Обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.
 - Построить модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузить и подготовьте данные.
2. Обучить разные модели. 
3. Сформулировать выводы.

## Подготовка
### Изучение данных
Импортируем необходимые инструменты.

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
import re
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mitman\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mitman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Загрузим данные.

In [2]:
try:    
    df = pd.read_csv('D:/notebook/project_13/toxic_comments.csv') 
except:     
    df = pd.read_csv('/datasets/toxic_comments.csv')

# Описание данных

In [3]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')] # удалим безымянный столбец 

In [5]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Проверим информацию о датафрейме.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


В датафреме содержится `159292` строка и `2` столбца:
 - Столбец `text` - текст комментария, тип данных object
 - Столбец `toxic` - токсичность комментария (целевой признак), тип данных `int64`
 
Преобразуем типы данных в столбце `'toxic'` для экономии места.

In [7]:
df.toxic = pd.to_numeric(df.toxic, downcast='integer')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 1.4+ MB


In [8]:
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Посмотрим долю токсичных к общему количеству комментариев.

In [9]:
round(sum(df['toxic']) / len(df), 2)

0.1

Подготовим данные для векторизации.
- Проведём лемматизацию слов с помощью WordNetLemmatizer() из библиотеки `nltk`
- Удалим пунктуацию и лишние пробелы
- Удалим стоп-слова (пока загрузим список, удалять будем в процессе tf-idf векторизации)

In [10]:
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [11]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return " ".join(text.split())

In [12]:
corpus = df['text']

In [13]:
corpus

0         Explanation\nWhy the edits made under my usern...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         "\nMore\nI can't make any real suggestions on ...
4         You, sir, are my hero. Any chance you remember...
                                ...                        
159287    ":::::And for the second time of asking, when ...
159288    You should be ashamed of yourself \n\nThat is ...
159289    Spitzer \n\nUmm, theres no actual article for ...
159290    And it looks like it was actually you who put ...
159291    "\nAnd ... I really don't think you understand...
Name: text, Length: 159292, dtype: object

In [14]:
%%time
corpus_lemm = [lemmatize(clear_text(corpus[i])) for i in range(len(corpus))]

Wall time: 1min 10s


In [15]:
corpus_lemm

['Explanation Why the edits made under my username Hardcore Metallica Fan were reverted They weren t vandalism just closure on some GAs after I voted at New York Dolls FAC And please don t remove the template from the talk page since I m retired now',
 'D aww He match this background colour I m seemingly stuck with Thanks talk January UTC',
 'Hey man I m really not trying to edit war It s just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page He seems to care more about the formatting than the actual info',
 'More I can t make any real suggestion on improvement I wondered if the section statistic should be later on or a subsection of type of accident I think the reference may need tidying so that they are all in the exact same format ie date format etc I can do that later on if no one else doe first if you have any preference for formatting style on reference or want to do it yourself please let me know There appears to be

In [16]:
stopwords = set(nltk_stopwords.words('english'))

Разобьем выборки на обучающую и тестовую.

In [17]:
X_train, X_test, y_train, y_test = train_test_split(corpus_lemm, df['toxic'], test_size=0.2, random_state=42)

Посмотрим на размеры выборок.

In [18]:
print(f"Размер обучающего корпуса: {len(X_train)}")
print(f"Размер тестового корпуса: {len(X_test)}")

Размер обучающего корпуса: 127433
Размер тестового корпуса: 31859


Выполним векторизацию корпусов с помощью TfidfVectorizer и удалим стоп-слова.

In [19]:
tf_idf_vec = TfidfVectorizer(ngram_range=(1,1), stop_words=stopwords,min_df=3, max_df=0.9,
                             use_idf=1, smooth_idf=1, sublinear_tf=1 )

Обучим модель без использования n-gramm

In [20]:
X_train_vec = tf_idf_vec.fit_transform(X_train)

In [21]:
X_test_vec = tf_idf_vec.transform(X_test)

In [22]:
print(f"Размер обучающего блока: {X_train_vec.shape}")
print(f"Размер тестового блока: {X_test_vec.shape}")

Размер обучающего блока: (127433, 41662)
Размер тестового блока: (31859, 41662)


## Обучение

Для константной модели найдём метрику accuracy. Предскажем все твиты нетоксичными ('toxic'=0)

In [23]:
base_predicts = pd.Series(data=np.zeros((len(y_test))), index=y_test.index, dtype='int16')
base_accuacy = accuracy_score(y_test, base_predicts)
print(f"Accuracy константной модели {base_accuacy:.3f}")

Accuracy константной модели 0.900


### Logistic Regression

- Обучим модель логистической регрессии. 
- Обучение, подбор гиперпараметров, кросс-валидацию проведём с помощью `GridSearchCV`
- Подбирать будем гиперпараметр регуляризации `С`

In [24]:
%%time

parameters = {'C': np.linspace(10, 20, num = 11, endpoint = True),
             'max_iter': [1000]}
lrm = LogisticRegression()
clf = GridSearchCV(lrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)
clf.fit(X_train_vec, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
Wall time: 1min 6s


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]),
                         'max_iter': [1000]},
             scoring='f1', verbose=2)

In [25]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.773
Параметр регуляризации для лучшей модели: {'C': 12.0, 'max_iter': 1000}


In [26]:
lrm = LogisticRegression(C=12, max_iter=1000)
lrm.fit(X_train_vec, y_train)
predict = lrm.predict(X_test_vec)
f1_lr = f1_score(y_test, predict)

In [27]:
print(f"Показатель f1 на тестовой выборке: {f1_lr:.3f}")

Показатель f1 на тестовой выборке: 0.775


Проверим модель на адекватность. Рассчитаем метрику accuracy и сравним её с константной моделью

In [28]:
accuracy_lr = accuracy_score(y_test, predict)
print(f"Accuracy на логистической регрессии {accuracy_lr:.3f}, больше, чем на константной модели")

Accuracy на логистической регрессии 0.960, больше, чем на константной модели


Показатель f1 на тестовой выборке удовлетворяет условию задачи.

### NB-SVM

Попробуем модель `NBSVM` (Naive Bayes - Support Vector Machine). В данной задаче будем испльзовать модель `LinearRegression` с алгоритмом оптимизации `solver='liblinear', dual=True`. В таком случае эти модели ведут себя похожим образом.

In [29]:
def prob(x, y_i, y):
    p = x[y==y_i].sum(axis=0)
    return (p+1) / ((y==y_i).sum()+1)

In [30]:
r = np.log(prob(X_train_vec, 1, y_train.values) / prob(X_train_vec, 0, y_train.values))
X_train_nb = X_train_vec.multiply(r)
X_test_nb = X_test_vec.multiply(r)

In [31]:
parameters = {'C': np.linspace(1, 11, num = 11, endpoint = True)}
nblrm = LogisticRegression(solver='liblinear', 
                           dual=True, 
                           max_iter = 1000)
clf_nb = GridSearchCV(nblrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)

In [32]:
clf_nb.fit(X_train_nb, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


GridSearchCV(cv=5,
             estimator=LogisticRegression(dual=True, max_iter=1000,
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])},
             scoring='f1', verbose=2)

In [33]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf_nb.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf_nb.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.791
Параметр регуляризации для лучшей модели: {'C': 3.0}


In [34]:
nblrm = LogisticRegression(C=3,
                           solver='liblinear', 
                           dual=True,
                           max_iter=1000)
nblrm.fit(X_train_nb, y_train)
predict = nblrm.predict(X_test_nb)
f1_nblr = f1_score(y_test, predict)

In [35]:
print(f"Показатель f1 на тестовой выборке: {f1_nblr:.3f}")

Показатель f1 на тестовой выборке: 0.788


- Данная модель мало отличается от изначальной Логистической регрессии (f1 вырос на 1,7%)
- Для сравнения попробуем обучить модель `LinearSVC`. Linear Support Vector Classification

### LinearSVC

In [36]:
parameters = {'C': np.linspace(1, 31, num = 7, endpoint = True)}
lsvcm = LinearSVC(max_iter = 1000)
clf_lsvc = GridSearchCV(nblrm, parameters,
                  cv=5,
                  scoring='f1',
                  n_jobs=-1,
                  verbose=2)

In [37]:
clf_lsvc.fit(X_train_vec, y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits


GridSearchCV(cv=5,
             estimator=LogisticRegression(C=3, dual=True, max_iter=1000,
                                          solver='liblinear'),
             n_jobs=-1,
             param_grid={'C': array([ 1.,  6., 11., 16., 21., 26., 31.])},
             scoring='f1', verbose=2)

In [38]:
print(f"Наилучший показатель f1 на кросс-валидации : {clf_lsvc.best_score_:.3f}")
print(f"Параметр регуляризации для лучшей модели: {clf_lsvc.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.773
Параметр регуляризации для лучшей модели: {'C': 16.0}


In [39]:
lsvcm = LogisticRegression(C=16,
                           max_iter=1000)
lsvcm.fit(X_train_vec, y_train)
predict = lsvcm.predict(X_test_vec)
f1_lsvc = f1_score(y_test, predict)

In [40]:
print(f"Показатель f1 на тестовой выборке: {f1_lsvc:.3f}")

Показатель f1 на тестовой выборке: 0.775


## Выводы

- Данные о токсичности твитов успешно загружены и обработаны:
    - Лемматизация проведена с помощью `WordNetLemmatizer` библиотеки `nltk`
    - Знаки пунктуации, а также лишние пробелы удалены
    - Стоп слова удалены (список взят из библиотеки `nltk`)
    - Корпус векторизован с помощью `TfidfVectorizer`
- На получившихся данных обучены модели, значение f1:
  - LogisticRegression  0.775     
  - NB-SVM              0.788     
  - LinearSVC           0.775     
  
- Качество моделей практически одинаково. Разница не более 1%. Максимальный показатель f1 получен для **NB-SVM: 0.788**
- Кросс-валидация моделей и подбор гиперпараметров проводились с помощью GridSearchCV.
- Проверка на адекватность была проведена для модели LogisticRegression.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны